# Iterativne metode
---

Za velike sustave, a posebno za sustave s malom ispunom (malo elemenata različitih od nule), te ukoliko je matrica sustava __strogo dijagonalno dominantna__, rješenje se može brzo naći __iterativnim metodama__
(vidi [Numerička matematika, poglavlje 3.8][RS04]):

__Definicija.__ Funkcija $F:\mathbb{R}^n\to \mathbb{R}^n$ je __kontrakcija__ ako postoji broj $q<1$ za koji vrijedi

$$
\| F(x)-F(y)\| < q\|x-y\|\qquad \forall x,y.
$$

__ Banachov teorem o fiksnoj točki.__ Ako je $F$ kontrakcija, tada niz definiran s

$$ 
x_{k+1}=F(x_k)
$$

konvergira prema jedinstvenom vektoru $\tilde x$ za kojeg vrijedi

$$
\tilde x = F(\tilde x).
$$

$\tilde x$ se zove __fiksna točka__ funkcije $F$. Za pogrešku u $k$-tom koraku vrijede ocjene

$$
\|x_k- \tilde x\| \leq \frac{q}{1-q} \|x_k-x_{k-1}\|
$$

i 

$$
\|x_k- \tilde x\| \leq \frac{q^k}{1-q} \|x_1-x_{0}\|,
$$

pri čemu je druga ocjena bolja. Brzina konvergencije je __linearna__,

$$
\|x_{k+1}-\tilde x\| \leq q\| x_k-\tilde x\|.
$$

[RS04]: http://www.mathos.unios.hr/pim/Materijali/Num.pdf "R. Scitovski, 'Numerička matematika', Sveučilište u Osijeku, Osijek, 2004."


## Jacobijeva i Gauss-Seidelova metoda

Neke je 

$$F(x)=Bx+c,$$

pri čemu je $B$ nesignularna matrica. Tada je

$$
\| F(x)-F(y)\|=\| Bx+c-(By+c)\|=\|B(x-y)\| \leq \|B\| \|x-y\|,
$$

pa je $F$ kontrakcija ako i samo ako je

$$
 \|B\|=q<1.
$$

Neka je zadan sustav  $Ax=b$. Matricu $A$ rastavimo kao

$$
A=D(L+I+U)
$$

pri čemu je $D$ dijagonalna matrica, $L$ strogo donje trokutasta matrica i $U$ strogo gornje trokutasta matrica.

__Jacobijeva metoda__:  Neka je 

$$
B=-(L+U), \quad c=D^{-1}b.
$$


Ako je matrica $A$ _strogo dijagonalno dominantna_, 

$$
\| B\|_{\infty} = \max_i \sum_{{j=1} \atop {j\neq i}}^n \frac{|a_{ij}|}{|a_{ii}|}<1,
$$

tada je preslikavanje $F$ kontrakcija (moguće je uzeti i druge norme).

Niz 

$$
x_{k+1}=-(L+U)x_k+c
$$

očito konvergira prema rješenju sustava $x$.

__Gauss-Seidel-ova metoda__:  Neka je 

$$
B=-(I+L)^{-1}U, \quad c=(I+L)D^{-1}b.
$$

> Bez dokaza navodimo sljedeću tvrdnju: ako je matrica $A$ strogo dijagonalno dominantna,
onda je preslikavanje $F$ kontrakcija pa

niz

$$
x_{k+1}=-(I+L)^{-1}Ux_k+(I+L)^{-1}D^{-1}b,
$$

odnosno

$$
x_{k+1}=-Lx_{k+1}-Ux_k+D^{-1}b,
$$

konvergira prema rješenju sustava $x$.

In [1]:
function myjacobi{T}(A::Array{T},b::Array{T},x::Array{T})
    D=diag(A)
    L=tril(A,-1)./D
    U=triu(A,1)./D
    tol=1000*eps()
    d=1.0
    B=-(L+U)
    c=b./D
    q=norm(B,Inf)
    # @show q
    while d>tol
        y=B*x+c
        d=norm(x-y,Inf)
        # @show d
        x=y
    end
    x,d
end

myjacobi (generic function with 1 method)

In [2]:
n=8
A=rand(n,n)
# Napravimo matricu dijagonalno diminantnom
A=A+n*I
b=rand(n)

8-element Array{Float64,1}:
 0.805988
 0.402526
 0.161728
 0.331226
 0.543806
 0.300015
 0.920605
 0.685248

In [3]:
# Pocetni vektor
x0=rand(n)

8-element Array{Float64,1}:
 0.910046 
 0.247026 
 0.134701 
 0.909162 
 0.133734 
 0.0525133
 0.517037 
 0.0477407

In [4]:
x,d=myjacobi(A,b,x0)

([0.07753311867593976,0.03550004464849055,0.0009601487497674296,0.027781229917682018,0.0432151734952167,0.01571276515024885,0.09252652473496706,0.07199801797186746],8.859579736508749e-14)

In [5]:
r=A*x-b

8-element Array{Float64,1}:
 -1.83187e-13
 -1.65701e-13
 -2.15827e-13
 -2.42084e-13
 -2.64677e-13
 -2.27929e-13
 -2.17382e-13
 -1.67755e-13

In [6]:
# Provjerimo i normu relativnog reziduala
norm(r)/(norm(A)*norm(x))

3.388749527609419e-13

In [7]:
function mygaussseidel{T}(A::Array{T},b::Array{T},x::Array{T})
    D=diag(A)
    L=tril(A,-1)./D
    U=triu(A,1)./D
    tol=1000*eps()
    d=1.0
    # B=-inv(I+L)*U
    B=-(I+L)\U
    c=(I+L)\(b./D)
    @show norm(U,Inf)
    y=Array(Float64,n)
    while d>tol
        y=B*x+c
        d=norm(x-y)
        x=y
    end
    x,d
end

mygaussseidel (generic function with 1 method)

In [8]:
x,d=mygaussseidel(A,b,x0)

norm(U,Inf) = 0.2932520086577795

([0.07753311867595566,0.035500044648507284,0.0009601487497859513,0.027781229917701676,0.043215173495238866,0.01571276515026942,0.09252652473498565,0.07199801797188302],3.0691503950227023e-14)

In [9]:
A*x-b

8-element Array{Float64,1}:
 -3.33067e-15
  1.83187e-14
  4.4964e-15 
 -7.88258e-15
 -5.66214e-15
 -1.33227e-15
  0.0        
  0.0        

Izmjerimo brzinu za veće matrice:

In [12]:
n=1024
A=rand(n,n)
# Napravimo matricu dijagonalno diminantnom
A=A+n*I
b=rand(n)
# Pocetni vektor
x0=rand(n)

1024-element Array{Float64,1}:
 0.378894  
 0.651432  
 0.630875  
 0.0203264 
 0.0175869 
 0.295135  
 0.0911703 
 0.702325  
 0.698973  
 0.0253697 
 0.22753   
 0.00961893
 0.509391  
 ⋮         
 0.283745  
 0.791041  
 0.673023  
 0.67891   
 0.820877  
 0.567777  
 0.366193  
 0.475563  
 0.467347  
 0.110154  
 0.116624  
 0.467151  

In [13]:
@time mygaussseidel(A,b,x0)

norm(U,Inf) = 0.507507065706234

([0.000176004,0.000673611,0.000664871,0.000158958,0.000601059,0.000379377,2.76895e-5,0.000582457,0.000544852,0.00064735  …  0.000169899,0.000649278,7.37901e-5,0.000390983,0.000781368,0.000497694,0.000323859,0.000179222,0.00062723,0.000788838],1.192690366098764e-13)


  0.128129 seconds (575 allocations: 80.429 MB, 9.72% gc time)
